<a href="https://colab.research.google.com/github/Nemczek/checkio_database/blob/main/pyCheckio_class_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project pyCheckio

This project downloads data about our class from [CheckIo](https://checkio.org) and stores it in SQLite database.

## Setting up

In [1]:
# importing libraries
import requests
import pandas as pd
from collections import Counter

In [2]:
# static variables
BASE_URL = 'https://py.checkio.org/api/group-details/'
GROUP_PROGRESS_API_BASE = 'https://py.checkio.org/api/group-progress/'
GROUP_ACTIVITY_API_BASE = 'https://py.checkio.org/api/group-activity/'
# Here you need to put your personal group token
TOKEN = None

URL_WITH_TOKEN = BASE_URL + TOKEN
PROGRESS_API_WITH_TOKEN = GROUP_PROGRESS_API_BASE + TOKEN
ACTIVITY_API_WITH_TOKEN = GROUP_ACTIVITY_API_BASE + TOKEN

In [ ]:
# listing all classes
requests.get(URL_WITH_TOKEN).json()['objects']

In [4]:
def get_slug(url):
  """
  Returns slag parameter of our current class

  Parameters:
  url (str): URL to our class details

  Returns:
  slug (str): the slug parameter
  """
  slug = requests.get(url).json()['objects'][4]['slug'] # 4 is the index of our class
  return slug
print(get_slug(URL_WITH_TOKEN))

michal-wojcik-2022-2023


## Activity API

In [5]:
# This API gives us acces to the latest activity of users
class_slug = get_slug(URL_WITH_TOKEN)
activity_url_with_slug = f"{ACTIVITY_API_WITH_TOKEN}&slug={class_slug}"

resp = requests.get(activity_url_with_slug).json()['objects']
resp[2]

{'username': '126086',
 'createdAt': '2023-01-09',
 'data': {'task': {'imageUrl': 'https://d17mnqrx9pmt3e.cloudfront.net/media/logos/task/normal/password-enabled.png',
   'shortText': 'Verify password by condition\n',
   'subject': 'Acceptable Password I',
   'type': 'task',
   'url': '/mission/acceptable-password-i/'},
  'type': 'implementation',
  'user': {'avatarUrl': 'https://www.gravatar.com/avatar/c13a3f1c3609338a3cc532e0eacc70eb?s=80',
   'level': 4,
   'username': '126086',
   'group': {'name': 'Michal Wojcik 2022-2023',
    'url': '/class/michal-wojcik-2022-2023/',
    'owner': 'MichalRyszardWojcik',
    'slug': 'michal-wojcik-2022-2023'},
   'type': 'user',
   'url': '/user/126086/'},
  'url': '/class/michal-wojcik-2022-2023/solution-history/3510415/'}}

## Progress API

This is main target of this project since all intresting data is here

In [6]:
# Get data from API
progress_url_with_slug = f"{PROGRESS_API_WITH_TOKEN}&slug={class_slug}"
progress_data = requests.get(progress_url_with_slug).json()['objects']

In [ ]:
progress_data[0] # Look on data

## Quest dataset

In [8]:
# unique statuses
statuses = []
for task in progress_data:
  for user in task['data']:
    statuses.append(user['status'])
set(statuses)

{'new', 'opened', 'published', 'tried'}

In [9]:
# Extract data about tasks solved by students to list of lists
list_of_tasks = []
for task in progress_data:
  num_of_votes = 0
  num_of_comments = 0
  num_of_tries = 0
  list_of_statuses = []

  for entry in task['data']:
    list_of_statuses.append(entry['status'])

    for solution in entry['solutions']:
      num_of_votes += solution['votes']
      num_of_comments += solution['comments']

  counter_object = Counter(list_of_statuses)
  list_of_tasks.append([task['title'], num_of_votes, num_of_comments,
                        counter_object['opened'], counter_object['published'],
                        counter_object['tried'], counter_object['new']])

In [ ]:
# Change list of lists to pandas DataFrame
task_data = pd.DataFrame(list_of_tasks, columns=['Task', 'Votes', 'Comments',
                                                 'Opened', 'Published', 'Tried',
                                                 'New'])
task_data

## Every user attempt dataset

In [11]:
# Extract data about every student's attempt to solve task to list of lists
list_of_entries = []

for task in progress_data:
  task_name = task['title']

  for entry in task['data']:
    username = entry['username']
    status = entry['status']

    if len(entry['solutions']) == 0:
      url, createdAt, votes, comments = "None", "None", "None", "None"
    else:
      # I'm taking only first solution
      url = entry['solutions'][0]['url']
      createdAt = entry['solutions'][0]['createdAt']
      votes = entry['solutions'][0]['votes']
      comments = entry['solutions'][0]['comments']

    list_of_entries.append([username, status, task_name, createdAt, votes, comments, url])


In [ ]:
# Convert to pandas data frame
entry_df = pd.DataFrame(list_of_entries, columns=['username', 'status',
                                                  'task_name', 'createdAt',
                                                  'votes', 'comments', 'url'])
entry_df

## Exporting data to SQL

In [13]:
%%capture
db_name = "checkio_class.db"

%load_ext sql
%sql sqlite:///{db_name}

import sqlalchemy as db
engine = db.create_engine(f'sqlite:///{db_name}')

entry_df.to_sql('entry_df', engine, index=False)
task_data.to_sql('task_data', engine, index=False)
# This code whill throw an error if database alredy exists.

In [ ]:
# Some tests to check if everything went well

import sqlite3 as sq
connection = sq.connect(db_name)
cursor = connection.cursor()

query = "SELECT * from task_data;"
result = cursor.execute(query)
rows = result.fetchall()

pd.DataFrame(rows, columns=map(lambda x: x[0], result.description))

In [ ]:
query2 = 'SELECT * FROM entry_df'
result2 = cursor.execute(query2)
rows2 = result2.fetchall()

pd.DataFrame(rows2, columns=map(lambda x: x[0], result2.description))

In [ ]:
query3 = "SELECT * FROM entry_df WHERE NOT votes = 'None' AND username = 'Antoni_Wojcik'"
result3 = cursor.execute(query3)
rows3 = result3.fetchall()

pd.DataFrame(rows3, columns=map(lambda x: x[0], result3.description))

****

# Upgrading database (Snowflake design)

## Extracting usernames to new table (Snowflake design)

In [17]:
query_create_username_table = """
CREATE TABLE users(
  	id INTEGER PRIMARY KEY AUTOINCREMENT,
  	name TEXT NOT NULL
)
"""
cursor.execute(query_create_username_table)

In [18]:
# fill users with data from entry_df database
query_fill_users = """
INSERT INTO users (name)
SELECT DISTINCT username
FROM entry_df
"""
cursor.execute(query_fill_users)

In [19]:
# Create username id in entry_df
query_add_id_column = """
ALTER TABLE entry_df
ADD COLUMN user_id INTEGER REFERENCES users (id) ON DELETE CASCADE
"""
cursor.execute(query_add_id_column)

In [20]:
# Add corresponding id's to entry_df
query_add_ids = """
UPDATE entry_df
SET user_id = (
  SELECT id
  FROM users
  WHERE name = username
)
"""
cursor.execute(query_add_ids)

In [21]:
# Drop username column from entry_df as it's no longer needed
query_drop_username = """
ALTER TABLE entry_df
DROP COLUMN username
"""
cursor.execute(query_drop_username)

## Testing - usernames table

In [22]:
# Sanity check -> table creation
queryt = 'SELECT * FROM sqlite_schema'
resultt = cursor.execute(queryt)
rowst = resultt.fetchall()

pd.DataFrame(rowst, columns=map(lambda x: x[0], resultt.description))

,type,name,tbl_name,rootpage,sql
0,table,entry_df,entry_df,2,"CREATE TABLE entry_df (\n\tstatus TEXT, \n\tta..."
1,table,task_data,task_data,237,"CREATE TABLE task_data (\n\t""Task"" TEXT, \n\t""..."
2,table,users,users,240,CREATE TABLE users(\n \tid INTEGER PRIMARY KE...
3,table,sqlite_sequence,sqlite_sequence,241,"CREATE TABLE sqlite_sequence(name,seq)"


In [26]:
# Sanity check -> names in users table
querytes = 'SELECT * FROM users'
resulttes = cursor.execute(querytes)
rowstes = resulttes.fetchall()

pd.DataFrame(rowstes, columns=map(lambda x: x[0], resulttes.description))

,id,name
0,1,karol2202
1,2,Antoni_Wojcik
2,3,117374
3,4,126212
4,5,Karolina_Zadura
5,6,126083
6,7,117370
7,8,Rafal.Keszycki
8,9,119587
9,10,Mariaa_Kuczynskaa


In [29]:
# Sanity check -> editing entry_df table
querytes = 'SELECT * FROM entry_df WHERE user_id = 3 LIMIT 5'
resulttes = cursor.execute(querytes)
rowstes = resulttes.fetchall()

pd.DataFrame(rowstes, columns=map(lambda x: x[0], resulttes.description))

,status,task_name,createdAt,votes,comments,url,user_id
0,published,Multiply (Intro),2022-11-21 15:31,0,0,https://py.checkio.org/mission/multiply-intro/...,3
1,published,Acceptable Password I,2022-12-12 20:42,0,0,https://py.checkio.org/mission/acceptable-pass...,3
2,published,Is Even,2022-11-21 19:04,0,0,https://py.checkio.org/mission/is-even/publica...,3
3,published,First Word (simplified),2022-11-21 19:00,0,0,https://py.checkio.org/mission/first-word-simp...,3
4,published,Number Length,2022-11-21 16:46,0,0,https://py.checkio.org/mission/number-length/p...,3


In [25]:
# Sanity check -> joining
querytes = """
SELECT * FROM entry_df
JOIN users
ON user_id = id
WHERE name = "117370"
"""
resulttes = cursor.execute(querytes)
rowstes = resulttes.fetchall()

pd.DataFrame(rowstes, columns=map(lambda x: x[0], resulttes.description))

,status,task_name,createdAt,votes,comments,url,user_id,id,name
0,published,Multiply (Intro),2022-11-28 14:57,0,1,https://py.checkio.org/mission/multiply-intro/...,7,7,117370
1,published,Acceptable Password I,2022-11-28 15:02,0,0,https://py.checkio.org/mission/acceptable-pass...,7,7,117370
2,published,Is Even,2022-11-28 15:10,0,0,https://py.checkio.org/mission/is-even/publica...,7,7,117370
3,published,First Word (simplified),2023-01-26 15:57,0,0,https://py.checkio.org/mission/first-word-simp...,7,7,117370
4,published,Number Length,2023-01-26 21:43,0,0,https://py.checkio.org/mission/number-length/p...,7,7,117370
...,...,...,...,...,...,...,...,...,...
243,new,Weak Point,None,None,None,None,7,7,117370
244,opened,Working Hours Calculator,None,None,None,None,7,7,117370
245,opened,Work Schedule Generator,None,None,None,None,7,7,117370
246,new,Xs and Os Champion,None,None,None,None,7,7,117370


## Extracting task_name to new table (Snowflake design)

In [30]:
query_create_task_table = """
CREATE TABLE tasks (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  task_name TEXT NOT NULL
)
"""
cursor.execute(query_create_task_table)

In [31]:
query_fill_task = """
INSERT INTO tasks (task_name)
SELECT DISTINCT task_name
FROM entry_df
"""
cursor.execute(query_fill_task)

In [36]:
query_alter_entry = """
ALTER TABLE entry_df
ADD COLUMN task_id INTEGER REFERENCES tasks (id) ON DELETE CASCADE
"""
cursor.execute(query_alter_entry)

In [44]:
query_add_ids_task = """
UPDATE entry_df
SET task_id = (
  SELECT id
  FROM tasks
  WHERE name = task_name
)
"""
cursor.execute(query_add_ids_task)

In [51]:
query_delete_task_name = """
ALTER TABLE entry_df
DROP COLUMN task_name
"""
cursor.execute(query_delete_task_name)

## Testing - tasks table

In [48]:
queryt = 'SELECT * FROM tasks'
resultt = cursor.execute(queryt)
rowst = resultt.fetchall()

pd.DataFrame(rowst, columns=map(lambda x: x[0], resultt.description))

,id,name
0,1,Multiply (Intro)
1,2,Acceptable Password I
2,3,Is Even
3,4,First Word (simplified)
4,5,Number Length
...,...,...
243,244,Weak Point
244,245,Working Hours Calculator
245,246,Work Schedule Generator
246,247,Xs and Os Champion


In [52]:
queryt = 'SELECT * FROM entry_df LIMIT 5'
resultt = cursor.execute(queryt)
rowst = resultt.fetchall()

pd.DataFrame(rowst, columns=map(lambda x: x[0], resultt.description))

,status,createdAt,votes,comments,url,user_id,task_id
0,published,None,None,None,None,1,1
1,published,2022-11-21 14:02,9,0,https://py.checkio.org/mission/multiply-intro/...,2,1
2,published,2022-11-21 15:31,0,0,https://py.checkio.org/mission/multiply-intro/...,3,1
3,published,None,None,None,None,4,1
4,published,2022-11-21 15:39,0,0,https://py.checkio.org/mission/multiply-intro/...,5,1


## Extracting statuses to new table (Snowflake design)

In [55]:
create_status_table = """
CREATE TABLE statuses (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT
)
"""
cursor.execute(create_status_table)

In [56]:
fill_statuses = """
INSERT INTO statuses (name)
SELECT DISTINCT status FROM entry_df
"""
cursor.execute(fill_statuses)

In [58]:
add_status_id = """
ALTER TABLE entry_df
ADD COLUMN status_id INTEGER REFERENCES statuses (id) ON DELETE CASCADE
"""
cursor.execute(add_status_id)

In [61]:
insert_status_id = """
UPDATE entry_df
SET status_id = (
  SELECT id
  FROM statuses
  WHERE name = status
)
"""
cursor.execute(insert_status_id)

In [65]:
delete_status = """
ALTER TABLE entry_df
DROP COLUMN status
"""
cursor.execute(delete_status)

## Testing - statuses

In [ ]:
queryt = 'SELECT * FROM entry_df'
resultt = cursor.execute(queryt)
rowst = resultt.fetchall()

pd.DataFrame(rowst, columns=map(lambda x: x[0], resultt.description))